<a href="https://colab.research.google.com/github/dohi1004/AI-teamProject-brailleTranslation/blob/main/AI_FinalProject_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# **AI Team Project Final Presentation**

### **Team 6**

16102269 Kim Jong Gyu

19102095 Lee Do Hui
***



# **1. Motivation For The Proeject**
* Similar to general language, braille is different according to the world.
* T

시각 장애인이 외국어로 되어 있는 점자를 읽어야 하는 경우 해당 나라 언어의 점자를 배운 적이 없다면 읽을 수 없습니다. 따라서, 외국어 점자를 모국어 음성으로 변환해주는 application을 만들고자 합니다.

# **2. Problem Description**
Here, we suppose the use case as follows. Here, we use similar word order languages (English, French) because of using basic NMT model.

1) English blind 

2) French

1. English blind write braille 
2. Translate English braille to English (with Braille Translation)
3. Translate English to French (with Neural Machine Translation)


**Braille Translation and Language Translation**

### 1) Braille Translation

Translate English braille to English.

### 2) Language Translation

Translate English to French.

**Finally, our application can translate other language's braille to target person's language by connecting those two models.** 



# **3. Data Description**

### **Braille Translation**
*   ~~Preferentially, I use braille notation in English dataset in Kaggle~~
  *   ~~Reference : https://www.kaggle.com/code/kwisatzhaderach/braille-classifier-keras/data~~

### **Neural Machine Translation**

*   For neural machine translation model, we used dataset comprised of French phrases and their English counterparts. The dataset is available from the http://www.manythings.org/anki/, with examples drawn from the Tatoeba Project.

<img src = "https://drive.google.com/uc?id=1zgHVKVLXsjyD4re6bWE5baV0q71J4yQo" height = 250 width = 500>

* The fra.txt file contains pairs of English to French phrases, one pair per line with a tab separating the language. Total pairs of sentences are 197,463.

<img src = "https://drive.google.com/uc?id=1-WO41TvUqFickYyjN298293hUtBSZRUs" height = 150 width = 600>

------------------
Preprocessing process of each data is explained in **5. Implementation Details**

------------------

# **4.Model Architecture**

### 1) Model 1: Convolutional Neural Network for Braille Translation

a model consisting of ~~~~> 

### 2) Model 2: Sequence-to-Sequence model - LSTM & GRU for Neural Machine Translation
**Encoder-Decoder based model**
다양한 길이의 input/output 을 동일한 길이의 memory에 할당한다.
-> 그래서 input 의 길이는 다양해도 상관 없음 (Maps variable-length sequence to fixed-length memory)
--------------------
* Input Language: English

* Output Language: French
---------------

1. Encoder

  It reads the input sequence and summarizes the information in one vector, called context vector (in LSTM, hidden state and cell state vectors). This context vector aims to contain all information for all input elements to help the decoder make accurate predictions. The hidden and cell state of the network is passed along to the decoder as input. 

2. Decoder

  It interprets the context vector obtained from the encoder. The context vecotr of the encoder's final cell is input to the first cell of the decoder network. Using these initial states, the decoder starts generating the output sequence, and these outputs are also considered for futrue predictions. 

  
Our problem is multiclass classification, so we use softmax activation function and cross entropy for the output layer. 

Our decoder_outputs aren't one-hot encoded, so they are integer labels. Therefore, we use sparse_categorical_crossentropy for loss function to solve the multi-class classification.



<img src = "https://drive.google.com/uc?id=1XChI2rJDoa6r8nL7hcBEELICLT-bDK8N" height = 300 width = 800>

----------------------


3) Final Architecture



<img src = "https://drive.google.com/uc?id=1PRGd8u2JUUeMloKnRxjW9zLl3WaDxJlu" height = 400 width = 850>

# **5. Implementation Details**

## **Part1. Braille Translation Model** 

##  **Part2. NMT model (English -> French)**


In [5]:
from google.colab import drive
drive.mount('/content/gdrive/') 

Mounted at /content/gdrive/


In [6]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

### **Data**

* Parallel corpus data 

- source: English

- target: French



In [3]:
length = 0
with open("./gdrive/MyDrive/AI/teamProject/fra.txt", "r", encoding='UTF-8') as lines:
        for i, line in enumerate(lines):
            length+=1
print(length)

197463


In [7]:
import pandas as pd
data = pd.read_csv("./gdrive/MyDrive/AI/teamProject/fra.txt", delimiter = "\t")
data.columns = ["en", "fr", "cc"]
data

,en,fr,cc
0,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
197457,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
197458,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
197459,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
197460,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


## **Data Preprocessing**

In [8]:
def to_ascii(s):
    # delete accent in French
    # EX) : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
    # Call accent deletion function
    sent = to_ascii(sent.lower())
    
    # Add blank between word and punctuation 
    # ex) "I am a student." => "I am a student ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)
    
    # Except (a-z, A-Z, ".", "?", "!", ","), transform others to blank
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    
    # Transform multiple blanks to one blank
    sent = re.sub(r"\s+", " ", sent)
    
    return sent

In [13]:
# Preprocessing Test
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('Before preprocessing English sentence :', en_sent)
print('After preprocessing English sentence :',preprocess_sentence(en_sent))
print('Before preprocessing French sentence :', fr_sent)
print('After preprocessing French sentence :', preprocess_sentence(fr_sent))

Before preprocessing English sentence : Have you had dinner?
After preprocessing English sentence : have you had dinner ?
Before preprocessing French sentence : Avez-vous déjà diné?
After preprocessing French sentence : avez vous deja dine ?


In [9]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    src_seqlen = 0
    tar_seqlen = 0
    with open("./gdrive/MyDrive/AI/teamProject/fra.txt", "r", encoding='UTF-8') as lines:
        for i, line in enumerate(lines):
            # split source data and target data 
            src_line, tar_line, _ = line.strip().split('\t')
            
            # preprocess source data
            preprocessed_sen = preprocess_sentence(src_line)
            src_line = [w for w in preprocessed_sen.split()]
            for w in preprocessed_sen.split():
              if len(w) > src_seqlen:
                src_seqlen = len(w)
            
            # preprocess target data -> we'll use teacher forcing in training phase, so divide decoder's input sequence and original value (label)
            tar_line = preprocess_sentence(tar_line)
            for w in tar_line.split():
              if len(w) > tar_seqlen:
                tar_seqlen = len(w)
            #  For target sequence (French) adding <sos> meaning start and <eos> meaning end.  
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]
            
            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)
            
            if i == num_samples - 1:
                break
        
    return encoder_input, decoder_input, decoder_target, src_seqlen, tar_seqlen

In [10]:
num_samples = 70000 # we use 70000 among about 190000 data

In [11]:
# Top 5 samples
sents_en_in, sents_fra_in, sents_fra_out, src_seqlen, tar_seqlen = load_preprocessed_data()
print('Input of Encoder :',sents_en_in[:5])
print('Input of Decoder :',sents_fra_in[:5])
print('Label of Decoder :',sents_fra_out[:5])
print(src_seqlen)
print(tar_seqlen)

Input of Encoder : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
Input of Decoder : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
Label of Decoder : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]
16
18


### Teacher Forcing

* Technique where the target word(ground truth) is passed as the next input to the decoder 


### Why it is needed?

* The problem with training recurrent neural networks that use output from prior time steps as input.

-> There is a risk that every single part part fails even though only one mistake in the first one. (previous decoder cell's prediction is wrong, this affects entire prediction wrong.) 

-> Analogy : a teacher records the score for each individual part and then tells the student the correct answre, to be used in the next part. (giving real value instead of predicted previous value.)

In [12]:
# Generate set of word with keras tokenizer and do integer encoding and padding 
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [17]:
# Shape of each data
print('Input of Encoder Shape :',encoder_input.shape)
print('Input of Decoder Shape :',decoder_input.shape)
print('Label of Decoder Shape :',decoder_target.shape)

Input of Encoder Shape : (70000, 9)
Input of Decoder Shape : (70000, 17)
Label of Decoder Shape : (70000, 17)


In [13]:
# The size of word set 
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("Size of English word set : {:d}, Size of French word set : {:d}".format(src_vocab_size, tar_vocab_size))

Size of English word set : 7241, Size of French word set : 12336


In [14]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [15]:
# Before spliting dataset, randomly mixing integer sequence list. 
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('Random Sequence :',indices)

Random Sequence : [29848 11081 63137 ... 22425  4389 22922]


In [16]:
# set it to dataset's order -> samples are mixed unlike original order 
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [17]:
# print randomly selected sample. 
# Here, decoder_input and decoder_target should have same integer sequence except <sos> and <eos> token.
print(encoder_input[40111])
print(decoder_input[40111])
print(decoder_target[40111])

[  6  84  11 290   5 389   1   0   0]
[  2  10  22   7 122 310   1   0   0   0   0   0   0   0   0   0   0]
[ 10  22   7 122 310   1   3   0   0   0   0   0   0   0   0   0   0]


In [18]:
# number of validation dataset (10%) 
n_of_val = int(num_samples*0.1)
print('Number of validation data :',n_of_val)

Number of validation data : 7000


In [19]:
# divide train (90%) and test data (10%)
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [20]:
# shape of traning and test data
print('Shape of training source data :',encoder_input_train.shape)
print('Shape of training target data :',decoder_input_train.shape)
print('Shape of trainig target label data :',decoder_target_train.shape)
print('Shape of test source data :',encoder_input_test.shape)
print('Shape of test target data :',decoder_input_test.shape)
print('Shape of test target label data :',decoder_target_test.shape)

Shape of training source data : (63000, 9)
Shape of training target data : (63000, 17)
Shape of trainig target label data : (63000, 17)
Shape of test source data : (7000, 9)
Shape of test target data : (7000, 17)
Shape of test target label data : (7000, 17)


## **LSTM model**

In [21]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [22]:
def LSTM_Model(embedding_dim, hidden_units, lr, b1, b2, batchsize, encoder_dropout, decoder_dropout):
  encoder_inputs = Input(shape=(None,))
  enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # Embedding layer
  enc_masking = Masking(mask_value=0.0)(enc_emb) # Exclude padding 0 in operation
  encoder_lstm = LSTM(hidden_units, return_state=True, dropout=encoder_dropout) # To return state value
  encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # Return hidden state and cell state
  encoder_states = [state_h, state_c] # Save encoder's hidden state and cell state -> to decoder!

  # Decoder
  decoder_inputs = Input(shape=(None,))
  dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # Embedding layer
  dec_emb = dec_emb_layer(decoder_inputs) # exclude padding 0 in operation
  dec_masking = Masking(mask_value=0.0)(dec_emb)

  # To return state value, return_state = True
  # To predict word for every time step, return_sequences = True
  decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True, dropout=decoder_dropout) 

  # Use encoder's hidden state as initial hidden state 
  decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                      initial_state=encoder_states)

  # predict word bsaed on softmax activation function for all results from every time step
  decoder_dense = Dense(tar_vocab_size, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)

  # Model's input and output 
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(optimizer=optimizers.Adam(learning_rate=lr, beta_1=b1, beta_2=b2), loss='sparse_categorical_crossentropy', metrics=['acc'])

  history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=batchsize, callbacks=[EarlyStopping(monitor='val_loss', patience = 3)], epochs=10) # for testing, we use epochs = 10 

  return history.history['val_loss'][-1]

#### **Hyperparameter Tuning**

In [ ]:
def get_hyper_param(n_iteration):
  hyper_param = []  # learning_rate, n_hidden, timestep, epochs
  for i in range(n_iteration):
    current_params = []
    # We use adam optimizer, so we tune learning rate, b1, b2
    current_params.append(np.random.uniform(0,0.1)) # learning rate 
    current_params.append(np.random.uniform(0.5,0.999)) # b1 (from Momentum)
    current_params.append(np.random.uniform(0.5,0.999)) # b2 (from RMSProp)
    current_params.append(np.random.randint(1,513)) # hidden units 
    current_params.append(np.random.randint(32,513)) # batch_size
    current_params.append(np.random.randint(50,300)) # embedding dimension
    current_params.append(np.random.uniform(0,0.5)) # encoder dropout
    current_params.append(np.random.uniform(0,0.5)) # decoder dropout
    hyper_param.append(current_params)
  return hyper_param

hyper_parameter = get_hyper_param(30)
train_loss, val_loss = list(), list()
best_params = []

min_val_loss = 999
for alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout in hyper_parameter:
  print('lr',alpha, 'b1', b1, 'b2', b2, 'n_hiddens', hidden_units,'batch_size', batch_size, 'embedding_dim', embedding_dim, 
        'en_dropout',  en_dropout, 'de_dropout', de_dropout)

  current_val_loss = LSTM_Model(embedding_dim, hidden_units, alpha, b1, b2, batch_size, en_dropout, de_dropout)

  if current_val_loss < min_val_loss:
    min_val_loss = current_val_loss
    best_params = [alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout]
    print('best_params',best_params)
  
print('final best params',best_params)

lr 0.06270461188149924 b1 0.843482643498749 b2 0.8925307649479214 n_hiddens 376 batch_size 455 embedding_dim 225 en_dropout 0.25149046826372107 de_dropout 0.28754374782314557
Epoch 1/10
139/139 [==============================] - 30s 161ms/step - loss: 1.9090 - acc: 0.7286 - val_loss: 1.3475 - val_acc: 0.7799
Epoch 2/10
139/139 [==============================] - 19s 140ms/step - loss: 1.2717 - acc: 0.7855 - val_loss: 1.2778 - val_acc: 0.7887
Epoch 3/10
139/139 [==============================] - 20s 141ms/step - loss: 1.2140 - acc: 0.7914 - val_loss: 1.2685 - val_acc: 0.7928
Epoch 4/10
139/139 [==============================] - 19s 140ms/step - loss: 1.1871 - acc: 0.7942 - val_loss: 1.2826 - val_acc: 0.7924
Epoch 5/10
139/139 [==============================] - 19s 139ms/step - loss: 1.1731 - acc: 0.7954 - val_loss: 1.2741 - val_acc: 0.7956
Epoch 6/10
139/139 [==============================] - 19s 139ms/step - loss: 1.1693 - acc: 0.7960 - val_loss: 1.2842 - val_acc: 0.7953
best_params [0.

### **Final LSTM model**

Based on best hyperparameter from hyperparameter tuning process, we build our final LSTM model with them. 

In [23]:
# learning rate, b1, b2, hidden_units, batch_size, embedding_dim, encoder dropout, decoder dropout
print('final best params',best_params)
LSTM_final_lr = best_params[0]
LSTM_final_b1 = best_params[1]
LSTM_final_b2 = best_params[2]
hidden_units = best_params[3]
LSTM_final_batch = best_params[4]
embedding_dim = best_params[5]
LSTM_final_enD = best_params[6]
LSTM_final_deD = best_params[7]

final best params [0.015886169959430787, 0.5930491086706786, 0.9364600597831465, 312, 273, 166, 0.1266038983112921, 0.47534027805897144]


In [2]:
best_params = [0.015886169959430787, 0.5930491086706786, 0.9364600597831465, 312, 273, 166, 0.1266038983112921, 0.47534027805897144]

In [24]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # Embedding layer
enc_masking = Masking(mask_value=0.0)(enc_emb) # Exclude padding 0 in operation
encoder_lstm = LSTM(hidden_units, return_state=True) # To return state value
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # Return hidden state and cell state
encoder_states = [state_h, state_c] # Save encoder's hidden state and cell state

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # Embedding layer
dec_emb = dec_emb_layer(decoder_inputs) # exclude padding 0 in operation
dec_masking = Masking(mask_value=0.0)(dec_emb)

# To return state value, return_state = True
# To predict word for every time step, return_sequences = True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# Use encoder's hidden state as initial hidden state 
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# predict word bsaed on softmax activation function for all results from every time step
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model's input and output 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=optimizers.Adam(learning_rate=LSTM_final_lr, beta_1=LSTM_final_b1, beta_2=LSTM_final_b2), 
              loss='sparse_categorical_crossentropy', metrics=['acc'])

model_weights_path = './gdrive/MyDrive/AI/teamProject/LSTM/weights/weights.ckpt'

save_best_weights = ModelCheckpoint(
  model_weights_path, monitor='val_loss', mode='min',
  save_weights_only=True, save_best_only=True, verbose=1, 
)

history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
        validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
        batch_size=LSTM_final_batch, callbacks=[save_best_weights,EarlyStopping(monitor='val_loss', patience = 10)], epochs=50) 

Epoch 1/50
  5/231 [..............................] - ETA: 14:58 - loss: 5.5656 - acc: 0.4773

KeyboardInterrupt: ignored

## **LSTM evaluation**

In NMT, there are diverse evaluation metrics from human evaluation methods to automatic evaluation metrics [1]. In previous diverse neural machine traslation tasks [2] [3], the mostly used evelaution metrics are BLEU score. Therefore, among diverse evlauation metrics, we use BLEU score and ROUGE score for our model's evaluation. 


<img src = "https://drive.google.com/uc?id=1JuvHUO16HlSU7N6B5zSPDOM3BVAKsXuL" height = 300 width = 400>


**BLEU Score [4] (n-gram precision)**

  BLEU is based on the degree of n-gram overlapping between the strings of words produced by the machine and the human translation references at the corpus level. BLEU computes the precision for n-gram of size 1-to-4 with
the coefficient of brevity penalty (BP).


**ROUGE Score [5] (n-gram recall)**

  ROUGE compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation. ROUGE is another metric that focuses upon recall rather than precision like BLEU.

* Diverse Variations, here we focus on ROUGE-1, ROUGE-2 and ROUGE-L

  1) ROUGE-N: how many n-grams from the reference translation have been properly predicted in the machine generated translation

  (ROUGE-1: unigrams, ROUGE-2: bigrams)

  2) ROUGE-L: measure of recall for longest common subsequences. 

---------------
BLEU focuses on precision: how much the words (and/or n-grams) in the candidate model outputs appear in the human reference.

ROUGE focuses on recall: how much the words (and/or n-grams) in the human references appear in the candidate model outputs.

These results are complementing, as is often the case in the precision-recall tradeoff. Therefore, we try to check both metrics for our model's evaluation.



In [15]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import nltk.translate.bleu_score as bleu
from rouge import Rouge

### Inference
seq2seq는 훈련 과정(교사 강요)과 테스트 과정에서의 동작 방식이 다릅니다. 그래서 테스트 과정을 위해 모델을 다시 설계해주어야 합니다. 특히 디코더를 수정해야 합니다. 이번에는 번역 단계를 위해 모델을 수정하고 동작시켜보겠습니다.
In sequence-to-sequence, the 

We design another model for test process, especially the decoder. 

The overall translation process is as follows. 

1) Input source sentence to be translated into encoder and get the final step's hidden and cell state. 

2) Encoder's final hidden state, cell state and \<sos> token is sent to decoder.

3) Decoder iterate next word prediction until \<eos> token.

4) The input and output of encoder, encoder_inputs and encoder_states is reused that was defined in training phase. 이렇게 되면 훈련 단계에 encoder_inputs와 encoder_states 사이에 있는 모든 층까지 전부 불러오게 되므로 결과적으로 훈련 단계에서 사용한 인코더를 그대로 재사용하게 됩니다. 

5) We control decoder's every step in test phase, so we define decoder_state_input_h, decoder_state_input_c which store previous step's state. 


In [ ]:
# Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder 
# Tensor for previous step's state 
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Reuse Embedding layer that was used in training phase 
dec_emb2 = dec_emb_layer(decoder_inputs)

# To predict next word.. 
# Previous time step's state -> current time step's initial state  
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# Predict word in every time step 
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Updated decoder 
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

테스트 단계에서의 동작을 위한 decode_sequence 함수를 구현합니다. 입력 문장이 들어오면 인코더는 마지막 시점까지 전개하여 마지막 시점의 은닉 상태와 셀 상태를 리턴합니다. 이 두 개의 값을 states_value에 저장합니다. 그리고 디코더의 초기 입력으로 <SOS>를 준비합니다. 이를 target_seq에 저장합니다. 이 두 가지 입력을 가지고 while문 안으로 진입하여 이 두 가지를 디코더의 입력으로 사용합니다. 이제 디코더는 현재 시점에 대해서 예측을 하게 되는데, 현재 시점의 예측 벡터가 output_tokens, 현재 시점의 은닉 상태가 h, 현재 시점의 셀 상태가 c입니다. 예측 벡터로부터 현재 시점의 예측 단어인 target_seq를 얻고, h와 c 이 두 개의 값은 states_value에 저장합니다. 그리고 while문의 다음 루프. 즉, 두번째 시점의 디코더의 입력으로 다시 target_seq와 states_value를 사용합니다. 이를 현재 시점의 예측 단어로 <eos>를 예측하거나 번역 문장의 길이가 50이 넘는 순간까지 반복합니다. 각 시점마다 번역된 다어는 decoded_sentence에 누적하여 저장하였다가 최종 번역 시퀀스로 리턴합니다.

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)
    
    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']
    
    stop_condition = False
    decoded_sentence = ''
    
    # Iterate loop until stop_condition becomes True
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char
        
        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True
            
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
        
    return decoded_sentence

결과 확인을 위한 함수를 만듭니다. seq_to_src 함수는 영어 문장에 해당하는 정수 시퀀스를 입력받으면 정수로부터 영어 단어를 리턴하는 index_to_src를 통해 영어 문장으로 변환합니다. seq_to_tar은 프랑스어에 해당하는 정수 시퀀스를 입력받으면 정수로부터 프랑스어 단어를 리턴하는 index_to_tar을 통해 프랑스어 문장으로 변환합니다.

In [ ]:
# transform original sentence's integer sequence to text sequence 
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence

# transform translated sentence's integer sequence to text sequence
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
            
    return sentence

### Evaluation Result

In [ ]:
rouge = Rouge()

for seq_index in [3, 50, 100, 300, 1001]:
    reference = [[]]
    candidate = []
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    

    input_sentence = seq_to_src(encoder_input_train[seq_index])
    answer_sentence = seq_to_tar(decoder_input_train[seq_index])
    translated_sentence = decoded_sentence[1:-5]

    print("Input Sentence :", input_sentence)
    print("Answer Sentence: ", answer_sentence)
    print("Translated Sentence :", translated_sentence)

    print(rouge.get_scores([translated_sentence], [answer_sentence], avg=True))
    reference[0].extend(answer_sentence.split())
    candidate.extend(translated_sentence.split())
    print('BLEU score: {:.5f}'.format(bleu.sentence_bleu(reference, candidate, weights=(1.0, 0, 0, 0))))
    
    print("-"*50)

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    
    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

[[ 21   3  33 136   4   0   0   0   0]]
1/1 [==============================] - 0s 16ms/step
입력문장 : are you all ready ? 
정답문장 : etes vous tous prets ? 
번역문장 : etes vous toutes dingues ? 
--------------------------------------------------
[[  2 634 336 280 305   1   0   0   0]]
1/1 [==============================] - 0s 17ms/step
입력문장 : i drank beer last night . 
정답문장 : j ai bu de la biere la nuit derniere . 
번역문장 : j ai la biere pour la biere . 
--------------------------------------------------
[[   8    9    5 2103  157    1    0    0    0]]
1/1 [==============================] - 0s 19ms/step
입력문장 : it s a sunny day . 
정답문장 : c est un jour ensoleille . 
번역문장 : c est une journee journee . 
--------------------------------------------------
[[  2  17  92  12 544   1   0   0   0]]
1/1 [==============================] - 0s 18ms/step
입력문장 : i m going to fight . 
정답문장 : je vais me battre . 
번역문장 : je vais etre prudente . 
--------------------------------------------------
[[   3   13 1931   

### **GRU model**

In [ ]:
from tensorflow.keras.layers import Input, Embedding, GRU
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [ ]:
def create_train_model(
  enc_seqlen, dec_seqlen, enc_vocabsize, dec_vocabsize, hsize, 
  embdim, encoder_dropout, decoder_dropout):
  ### Encoder
  encoder_inputs = Input(shape=(None,), name="encoder_input")
  embedding = Embedding(
      enc_vocabsize, embdim, name="encoder_embedding"
  )
  encoder_gru = GRU(
      hsize, return_state=True, name="encoder_gru", dropout=encoder_dropout
  )
  encoder_emb = embedding(encoder_inputs)
  encoder_outputs, encoder_state = encoder_gru(encoder_emb)

  ### Decoder
  decoder_inputs = Input(shape=(None,), name="decoder_input")
  embedding = Embedding(
      dec_vocabsize, embdim, input_length=dec_seqlen-1, name="decoder_embedding"
   )
  decoder_gru = GRU(
      hsize, return_state=True, return_sequences=True, name="decoder_gru", 
      dropout=decoder_dropout
  )

  decoder_emb = embedding(decoder_inputs)
  decoder_outputs, _ = decoder_gru(decoder_emb, initial_state=encoder_state)
  dense_layer = Dense(dec_vocabsize, activation="softmax", name="dense_layer")
  decoder_outputs = dense_layer(decoder_outputs)

  # Define the Model which accepts encoder/decoder inputs and outputs predictions
  model = Model(
      inputs=[encoder_inputs, decoder_inputs],
      outputs=decoder_outputs,
      name="encoder_decoder_model_gru",
  )
  return model

In [ ]:
def GRU_Model(embedding_dim, hidden_units, lr, b1, b2, batchsize, encoder_dropout, decoder_dropout):
  model = create_train_model(
    src_seqlen, tar_seqlen, src_vocab_size, tar_vocab_size, 
    hidden_units, embedding_dim, 
    encoder_dropout, decoder_dropout
  )

  model.compile(optimizer=optimizers.Adam(learning_rate=lr, beta_1=b1, beta_2=b2), loss='sparse_categorical_crossentropy', metrics=['acc'])

  history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=batchsize, callbacks=[EarlyStopping(monitor='val_loss', patience = 3)], epochs=10) # for testing, we use epochs = 10 


  return history.history['val_loss'][-1]

#### **Hyperparameter Selection Process**

Similar to LSTM, we try hyperparameter tuning to find best hyperparmeter for GRU.

In [ ]:
def get_hyper_param(n_iteration):
  hyper_param = []  # learning_rate, n_hidden, timestep, epochs
  for i in range(n_iteration):
    current_params = []
    # We use adam optimizer, so we tune learning rate
    current_params.append(np.random.uniform(0,0.1)) # learning rate 
    current_params.append(np.random.uniform(0.5,0.999)) # b1 (from Momentum)
    current_params.append(np.random.uniform(0.5,0.999)) # b2 (from RMSProp)
    current_params.append(np.random.randint(1,513)) # hidden units 
    current_params.append(np.random.randint(32,513)) # batch_size
    current_params.append(np.random.randint(50,300)) # embedding dimension
    current_params.append(np.random.uniform(0,0.5)) # encoder dropout
    current_params.append(np.random.uniform(0,0.5)) # decoder dropout
    hyper_param.append(current_params)
  return hyper_param

hyper_parameter = get_hyper_param(30)
train_loss, val_loss = list(), list()
best_params = []

min_val_loss = 999
for alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout in hyper_parameter:
  print('lr',alpha, 'b1', b1, 'b2', b2, 'n_hiddens', hidden_units,'batch_size', batch_size, 'embedding_dim', embedding_dim, 
        'en_dropout',  en_dropout, 'de_dropout', de_dropout)

  current_val_loss = GRU_Model(embedding_dim, hidden_units, alpha, b1, b2, batch_size, en_dropout, de_dropout)

  if current_val_loss < min_val_loss:
    min_val_loss = current_val_loss
    best_params = [alpha, b1, b2, hidden_units, batch_size, embedding_dim, en_dropout, de_dropout]

lr 0.015976380574435466 b1 0.6780522443921471 b2 0.9702656093996469 n_hiddens 153 batch_size 204 embedding_dim 137 en_dropout 0.25427391287383155 de_dropout 0.2550421119727371
Epoch 1/10
309/309 [==============================] - 24s 51ms/step - loss: 1.3410 - acc: 0.7907 - val_loss: 0.9090 - val_acc: 0.8371
Epoch 2/10
309/309 [==============================] - 15s 50ms/step - loss: 0.7851 - acc: 0.8480 - val_loss: 0.7665 - val_acc: 0.8563
Epoch 3/10
309/309 [==============================] - 15s 49ms/step - loss: 0.6456 - acc: 0.8643 - val_loss: 0.7159 - val_acc: 0.8652
Epoch 4/10
309/309 [==============================] - 15s 49ms/step - loss: 0.5727 - acc: 0.8731 - val_loss: 0.6909 - val_acc: 0.8682
Epoch 5/10
309/309 [==============================] - 15s 49ms/step - loss: 0.5285 - acc: 0.8786 - val_loss: 0.6812 - val_acc: 0.8710
Epoch 6/10
309/309 [==============================] - 15s 49ms/step - loss: 0.5010 - acc: 0.8820 - val_loss: 0.6705 - val_acc: 0.8732
Epoch 7/10
309/309 [

### **Final GRU model**

With final best hyperparameter for GRU model, we build our final GRU model based on those hyperparameter. 

In [ ]:
# learning rate, b1, b2, hidden_units, batch_size, embedding_dim, encoder dropout, decoder dropout
print('final best params',best_params)

GRU_final_lr = best_params[0]
GRU_final_b1 = best_params[1]
GRU_final_b2 = best_params[2]
GRU_hidden_units = best_params[3]
GRU_final_batch = best_params[4]
GRU_embedding_dim = best_params[5]
GRU_final_enD = best_params[6]
GRU_final_deD = best_params[7]

final best params [0.015976380574435466, 0.6780522443921471, 0.9702656093996469, 153, 204, 137, 0.25427391287383155, 0.2550421119727371]


In [ ]:
best_params = [0.015976380574435466, 0.6780522443921471, 0.9702656093996469, 153, 204, 137, 0.25427391287383155, 0.2550421119727371]

In [ ]:
def Final_GRU_Model(embedding_dim, hidden_units, lr, b1, b2, batchsize, encoder_dropout, decoder_dropout):
  model = create_train_model(
    src_seqlen, tar_seqlen, src_vocab_size, tar_vocab_size, 
    hidden_units, embedding_dim, 
    encoder_dropout, decoder_dropout
  )

  model.compile(optimizer=optimizers.Adam(learning_rate=lr, beta_1=b1, beta_2=b2), loss='sparse_categorical_crossentropy', metrics=['acc'])

  model_weights_path_gru = './gdrive/MyDrive/AI/teamProject/GRU/weights/weights.ckpt'

  save_best_weights = ModelCheckpoint(
    model_weights_path_gru, monitor='val_loss', mode='min',
    save_weights_only=True, save_best_only=True, verbose=1, 
  )

  history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=batchsize, callbacks=[EarlyStopping(monitor='val_loss', patience = 10)], epochs=50)


  return model

In [ ]:
model = Final_GRU_Model(GRU_embedding_dim, GRU_hidden_units, GRU_final_lr, GRU_final_b1, GRU_final_b2, GRU_final_batch, GRU_final_enD, GRU_final_deD)

Epoch 1/50
309/309 [==============================] - 19s 51ms/step - loss: 1.3089 - acc: 0.7944 - val_loss: 0.8632 - val_acc: 0.8441
Epoch 2/50
309/309 [==============================] - 15s 49ms/step - loss: 0.7337 - acc: 0.8552 - val_loss: 0.7159 - val_acc: 0.8647
Epoch 3/50
309/309 [==============================] - 16s 51ms/step - loss: 0.5901 - acc: 0.8728 - val_loss: 0.6697 - val_acc: 0.8718
Epoch 4/50
309/309 [==============================] - 16s 50ms/step - loss: 0.5242 - acc: 0.8809 - val_loss: 0.6556 - val_acc: 0.8755
Epoch 5/50
309/309 [==============================] - 16s 50ms/step - loss: 0.4875 - acc: 0.8854 - val_loss: 0.6548 - val_acc: 0.8755
Epoch 6/50
309/309 [==============================] - 15s 50ms/step - loss: 0.4631 - acc: 0.8886 - val_loss: 0.6538 - val_acc: 0.8764
Epoch 7/50
309/309 [==============================] - 15s 50ms/step - loss: 0.4479 - acc: 0.8906 - val_loss: 0.6508 - val_acc: 0.8769
Epoch 8/50
309/309 [==============================] - 15s 50ms

## Inference

In [ ]:
def create_inference_encoder_from(input_layer, embedding_layer, gru_layer):
  encoder_embedding = embedding_layer(input_layer)
  encoder_outputs, encoder_state = gru_layer(encoder_embedding)
  encoder = Model(input_layer, encoder_state)
  return encoder

def create_inference_decoder_from(input_layer, embedding_layer, gru_layer, dense_layer):
  # decoder_input = Input(shape=(1,))
  decoder_embedding = embedding_layer(input_layer)
  input_shape = dense_layer.input.shape[-1]
  decoder_inputs_state = Input(shape=(input_shape,))

  decoder_output, decoder_output_state = gru_layer(
    decoder_embedding, initial_state=decoder_inputs_state
  )

  decoder_prediction = dense_layer(decoder_output)
  decoder = Model(
    inputs=[input_layer, decoder_inputs_state], 
    outputs=[decoder_prediction, decoder_output_state]
  )
  return decoder

In [ ]:
encoder = create_inference_encoder_from(
  model.get_layer('encoder_input').input,
  model.get_layer('encoder_embedding'),
  model.get_layer('encoder_gru')
)

decoder = create_inference_decoder_from(
  model.get_layer('decoder_input').input,
  model.get_layer('decoder_embedding'),
  model.get_layer('decoder_gru'),
  model.get_layer('dense_layer'),
)

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    state = encoder.predict(input_seq)
    
    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']
    
    stop_condition = False
    decoded_sentence = ''
    
    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, state = decoder.predict([target_seq, state])
        
        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char
        
        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True
            
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        state = [state]
        
    return decoded_sentence

In [ ]:
#######     

### We set random parameter for comparing LSTM and GRU model. (here, same parameter is used for both models.)

# hidden_units = 64, embedding_dim = 64, batch_size = 128, 50_epoch 

## **Predicted Result & Evaluation**

### Evaluation Metrics
Same metrics as LSTM.

In [ ]:
import nltk.translate.bleu_score as bleu
from rouge import Rouge

In [ ]:
rouge = Rouge()

for seq_index in [3, 50, 100, 300, 1001]:
    reference = [[]]
    candidate = []
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    

    input_sentence = seq_to_src(encoder_input_train[seq_index])
    answer_sentence = seq_to_tar(decoder_input_train[seq_index])
    translated_sentence = decoded_sentence[1:-5]

    print("Input Sentence :", input_sentence)
    print("Answer Sentence: ", answer_sentence)
    print("Translated Sentence :", translated_sentence)

    print(rouge.get_scores([translated_sentence], [answer_sentence], avg=True))
    reference[0].extend(answer_sentence.split())
    candidate.extend(translated_sentence.split())
    print('BLEU score: {:.5f}'.format(bleu.sentence_bleu(reference, candidate, weights=(1.0, 0, 0, 0))))
    
    print("-"*50)

1/1 [==============================] - 0s 17ms/step
Input Sentence : we had a little water . 
Answer Sentence:  nous disposions d un peu d eau . 
Translated Sentence : on a eu un peu d eau . 
{'rouge-1': {'r': 0.7142857142857143, 'p': 0.625, 'f': 0.6666666616888889}, 'rouge-2': {'r': 0.5714285714285714, 'p': 0.5714285714285714, 'f': 0.5714285664285715}, 'rouge-l': {'r': 0.7142857142857143, 'p': 0.625, 'f': 0.6666666616888889}}
BLEU score: 0.62500
--------------------------------------------------
1/1 [==============================] - 0s 15ms/step
Input Sentence : who wants to go ? 
Answer Sentence:  qui veut y aller ? 
Translated Sentence : qui veut aller ? 
{'rouge-1': {'r': 0.8, 'p': 1.0, 'f': 0.8888888839506174}, 'rouge-2': {'r': 0.5, 'p': 0.6666666666666666, 'f': 0.5714285665306124}, 'rouge-l': {'r': 0.8, 'p': 1.0, 'f': 0.8888888839506174}}
BLEU score: 0.77880
--------------------------------------------------
1/1 [==============================] - 0s 15ms/step


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1/1 [==============================] - 0s 18ms/step
Input Sentence : who made this pie ? 
Answer Sentence:  qui a fait cette tourte ? 
Translated Sentence : qui a fait cette tarte ? 
{'rouge-1': {'r': 0.8333333333333334, 'p': 0.8333333333333334, 'f': 0.8333333283333335}, 'rouge-2': {'r': 0.6, 'p': 0.6, 'f': 0.5999999950000001}, 'rouge-l': {'r': 0.8333333333333334, 'p': 0.8333333333333334, 'f': 0.8333333283333335}}
BLEU score: 0.83333
--------------------------------------------------
1/1 [==============================] - 0s 15ms/step
Input Sentence : it feels like rain . 
Answer Sentence:  on dirait qu il va pleuvoir . 
Translated Sentence : il semble qu il pleuve . 
{'rouge-1': {'r': 0.42857142857142855, 'p': 0.6, 'f': 0.499999995138889}, 'rouge-2': {'r': 0.16666666666666666, 'p': 0.2, 'f': 0.18181817685950424}, 'rouge-l': {'r': 0.42857142857142855, 'p': 0.6, 'f': 0.499999995138889}}
BLEU score: 0.42324
--------------------------------------------------
1/1 [=========================

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


# **Final Result and Insights**

Our final application is making voice with translated text.

In [5]:
pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.2 MB/s 


In [1]:
 pip install gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 561 kB/s 
     |████████████████████████████████| 96 kB 3.3 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [6]:
from gtts import gTTS
import pygame

pygame 2.1.2 (SDL 2.0.16, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [14]:
text ="Hi, everybody. Playing with Python is fun!!!" # put translated text here

tts = gTTS(text=text, lang='en')
tts.save("helloEN.mp3")

In [8]:
pygame.mixer.init()
pygame.mixer.music.load('./helloEN.mp3')
pygame.mixer.music.play()

error: ignored

# **References**

[1] Han, Lifeng. "Machine translation evaluation resources and methods: A survey." arXiv preprint arXiv:1605.04515 (2016).

[2] Chimalamarri, Santwana, and Dinkar Sitaram. "Linguistically enhanced word segmentation for better neural machine translation of low resource agglutinative languages." International Journal of Speech Technology 24.4 (2021): 1047-1053.

[3] Wu, Yonghui, et al. "Google's neural machine translation system: Bridging the gap between human and machine translation." arXiv preprint arXiv:1609.08144 (2016).

[4] Papineni, Kishore, et al. "Bleu: a method for automatic evaluation of machine translation." Proceedings of the 40th annual meeting of the Association for Computational Linguistics. 2002.

[5] Chin-Yew Lin. 2004. ROUGE: A Package for Automatic Evaluation of Summaries. In Text Summarization Branches Out, pages 74–81, Barcelona, Spain. Association for Computational Linguistics.



# **Member's contribution statement**

# **Debugging experience worth sharing**

# **The Github repository with the commit history**